In [ ]:
pip install torch diffusers opencv-python pillow


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
from diffusers import StableDiffusionInpaintPipeline
from PIL import Image
import cv2
import numpy as np

# Load the Stable Diffusion inpainting pipeline
pipe = StableDiffusionInpaintPipeline.from_pretrained("runwayml/stable-diffusion-inpainting", torch_dtype=torch.float16)
pipe = pipe.to("cuda")


# Load source (face) and target (body) images
source_image_path = "source_face.jpg"  # Path to the source face image
target_image_path = "target_body.jpg"  # Path to the target body image

source_img = Image.open('/content/drive/MyDrive/Colab Notebooks/Face_Swap/source_face.jpg').convert("RGB")
target_img = Image.open('/content/drive/MyDrive/Colab Notebooks/Face_Swap/target_body.jpg').convert("RGB")

# Detect the face in the target image using OpenCV
target_cv_img = np.array(target_img)
gray = cv2.cvtColor(target_cv_img, cv2.COLOR_BGR2GRAY)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
faces = face_cascade.detectMultiScale(gray, 1.1, 4)

if len(faces) == 0:
    raise Exception("No face detected in the target image.")

# For simplicity, assuming only one face in the target image
(x, y, w, h) = faces[0]

# Resize the source image to match the detected face size
source_resized = source_img.resize((w, h), Image.Resampling.LANCZOS)

# Ensure the mask has the same mode and size as the target image
mask = Image.new("L", target_img.size, 0)
mask.paste(255, (x, y, x + w, y + h))

# Convert mask to match the source_resized image
mask_resized = mask.crop((x, y, x + w, y + h))

# Paste the resized face onto the target image
target_img.paste(source_resized, (x, y, x + w, y + h), mask_resized)

# Inpainting with Stable Diffusion
output = pipe(prompt="", image=target_img, mask_image=mask).images[0]

# Save the final image
output.save("output_image.jpg")

# Display the result
output.show()


safety_checker/model.safetensors not found


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280

  0%|          | 0/50 [00:00<?, ?it/s]